In [ ]:
from sklearn.metrics import classification_report, precision_recall_fscore_support, f1_score
from statsmodels.stats.inter_rater import fleiss_kappa
import pandas as pd
import numpy as np
import warnings
import re
import os
from tools import fleiss_pivot

from IPython.display import display_markdown

In [ ]:
def to_bool(string:str) -> bool:
    string = str(string).strip().casefold()
    if string == 'nan':
        return False
    if string == 'true':
        return True
    if string == 'false':
        return False
    
    # some random stackoverflow said not not is faster than bool()
    return not not string 

In [ ]:
IRR_FILES = ['./pass_1.5_roberta_400/leon.csv', './pass_1.5_roberta_400/roberta.csv']
KEY_COLUMNS = ['Date','Publication','Headline','URL']
LABELS = set()

# Create empty key dataframe
key = pd.DataFrame(columns=KEY_COLUMNS)
raters = dict()
for path in IRR_FILES:
    df = pd.read_csv(path)
    rater = os.path.basename(path).split('.')[0]
    
    # TODO: depending on our format, we will need to cast certain columns to bool
    
    raters[rater] = df

# Truncate all dataframes to have the same keys
all_keys = raters[0].index
for rater in raters:
    all_keys = all_keys.intersection(raters[rater].index)
for rater in raters:
    raters[rater] = raters[rater].loc[all_keys]

In [ ]:
# Classification report for a row that takes on two OR MORE values
# ex: "Category": 'A' or 'B' or 'C'
REPORT_COLUMN = 'Category'
rater1 = list(raters.keys())[0]
rater2 = list(raters.keys())[1]

print(
    classification_report(
        raters[rater1][REPORT_COLUMN],
        raters[rater2][REPORT_COLUMN],
    )
)
category_report = classification_report(
    raters[rater1][REPORT_COLUMN],
    raters[rater2][REPORT_COLUMN],
    output_dict=True
)

category_irr = fleiss_kappa(
    fleiss_pivot([
        raters[rater1][REPORT_COLUMN],
        raters[rater2][REPORT_COLUMN],
    ],
    REPORT_COLUMN)
)

print("Fleiss:", category_irr, sep=' ')

In [ ]:
# Create a markdown table for every label
rater1 = list(raters.keys())[0]
rater2 = list(raters.keys())[1]

f1_scores = list()
irr_scores = list()
md = '| Label | IRR | Precision | Recall | F1 | Support | In Agreement | Disagreement |\n|---|---|---|---|---|---|---|---|\n'
for label in LABELS:
    if label=='Category':
        continue
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        irr = fleiss_kappa(fleiss_pivot([
            raters[rater1][REPORT_COLUMN],
            raters[rater2][REPORT_COLUMN],
        ], label))
        p, r, f, s = precision_recall_fscore_support(raters[rater1][label].fillna(False), raters[rater2][label].fillna(False), average='binary')
    
    # Overwrite with support (max of both)
    s = max(raters[rater1][label].sum(), raters[rater2][label].sum())
    
    agree = (raters[rater1][label] == raters[rater2][label]).sum()
    disagree = len(key) - agree
    
    p, r, f, s = float(p), float(r), float(f), int(s) if s is not None else 0
    md += f'| {label} | {irr:.2f} | {p:.2f} | {r:.2f} | {f:.2f} | {s} | {agree} | {disagree} |\n'
    f1_scores.append(f)
    irr_scores.append(irr)
    
display_markdown(md, raw=True)
print('Average F1:', np.nanmean(f1_scores), sep=' ')
print('Average IRR:', np.nanmean(irr_scores), sep=' ')

In [ ]:
# Calculate overall Avergage F1 and Avergage IRR
category_f1 = category_report['macro avg']['f1-score']
category_irr = category_irr
labels_f1 = np.nanmean(f1_scores)
labels_irr = np.nanmean(irr_scores)

ncat = max(raters[rater1][REPORT_COLUMN].nunique(), raters[rater2][REPORT_COLUMN].nunique())

print('Category F1:', category_f1, sep=' ')
print('Category IRR:', category_irr, sep=' ')
print('Labels F1:', labels_f1, sep=' ')
print('Labels IRR:', labels_irr, sep=' ')
print('Overall F1:', (ncat * category_f1 + len(LABELS) * labels_f1) / (ncat + len(LABELS)), sep=' ')
print('Overall IRR:', (ncat * category_irr + len(LABELS) * labels_irr) / (ncat + len(LABELS)), sep=' ')